In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-may-2022/train.csv
/kaggle/input/tabular-playground-series-may-2022/test.csv


In [2]:
# Memory management 
import gc 

#modelling
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

In [3]:
# Load date sets.

train_data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')

test_data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')
test_ids = test_data['id'] 

# target


train_data.head()


,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,target
0,0,-1.373246,0.238887,-0.243376,0.567405,-0.647715,0.839326,0.113133,1,5,...,-2.540739,0.766952,-2.730628,-0.208177,1.363402,ABABDADBAB,67.609153,0,0,0
1,1,1.697021,-1.710322,-2.230332,-0.545661,1.113173,-1.552175,0.447825,1,3,...,2.278315,-0.633658,-1.217077,-3.782194,-0.058316,ACACCADCEB,377.096415,0,0,1
2,2,1.681726,0.616746,-1.027689,0.810492,-0.609086,0.113965,-0.708660,1,0,...,-1.385775,-0.520558,-0.009121,2.788536,-3.703488,AAAEABCKAD,-195.599702,0,2,1
3,3,-0.118172,-0.587835,-0.804638,2.086822,0.371005,-0.128831,-0.282575,3,2,...,0.572594,-1.653213,1.686035,-2.533098,-0.608601,BDBBAACBCB,210.826205,0,0,1
4,4,1.148481,-0.176567,-0.664871,-1.101343,0.467875,0.500117,0.407515,3,3,...,-3.912929,-1.430366,2.127649,-3.306784,4.371371,BDBCBBCHFE,-217.211798,0,1,1


In [4]:
#check for unique values
for col in test_data:
    print(col)
    print(test_data[col].unique())

id
[ 900000  900001  900002 ... 1599997 1599998 1599999]
f_00
[ 0.44251689 -0.60559756  0.30398973 ... -0.33170369 -2.03107297
 -0.08590636]
f_01
[ 1.74380235e-01 -3.05714509e-01  2.44511024e+00 ... -3.28844814e-01
 -1.23839802e+00 -2.12377493e-03]
f_02
[-0.99981593  0.62766725  0.24651454 ... -1.18550262  0.96469876
  2.22737467]
f_03
[ 0.76274101 -0.57889756  0.81824834 ...  1.02212795 -1.04595013
  0.21714491]
f_04
[ 0.18677843 -1.75093096  0.35973054 ... -0.48309949  0.90606413
  3.17915307]
f_05
[-1.0747749   1.35555045 -1.33184544 ... -0.10714629  0.63430066
 -1.66018827]
f_06
[ 0.50188752 -0.19091089  1.35862229 ... -0.96828118 -0.70747429
  0.89198907]
f_07
[ 6  1  3  0  2  5  8  4  7 12  9 10 11 13 15 16]
f_08
[ 6  3  0  2  1  4  5  8 11  7 10  9 12 13]
f_09
[ 0  4  2  3  9  5  1  6  7  8 10 12 11 13 16 14]
f_10
[ 0  1  2  4  3  6  5  7  8  9 10 12 11 14 15 13]
f_11
[ 0  2  1  3  5  4  7  6  8  9 11 10 12 13 14]
f_12
[ 5  1  3  4  6  2  7  0  9  8 10 11 12 13 14 15]
f_13
[ 1  

In [5]:
#look for any NAN values and types

train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900000 entries, 0 to 899999
Data columns (total 33 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      900000 non-null  int64  
 1   f_00    900000 non-null  float64
 2   f_01    900000 non-null  float64
 3   f_02    900000 non-null  float64
 4   f_03    900000 non-null  float64
 5   f_04    900000 non-null  float64
 6   f_05    900000 non-null  float64
 7   f_06    900000 non-null  float64
 8   f_07    900000 non-null  int64  
 9   f_08    900000 non-null  int64  
 10  f_09    900000 non-null  int64  
 11  f_10    900000 non-null  int64  
 12  f_11    900000 non-null  int64  
 13  f_12    900000 non-null  int64  
 14  f_13    900000 non-null  int64  
 15  f_14    900000 non-null  int64  
 16  f_15    900000 non-null  int64  
 17  f_16    900000 non-null  int64  
 18  f_17    900000 non-null  int64  
 19  f_18    900000 non-null  int64  
 20  f_19    900000 non-null  float64
 21  f_20    90

In [6]:
# check for dups Row

train_data[train_data.duplicated()]

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,target


In [7]:
# column duplicates

train_data.apply(lambda x : x.duplicated().sum())

id             0
f_00           0
f_01           0
f_02           0
f_03           0
f_04           0
f_05           0
f_06           0
f_07      899984
f_08      899984
f_09      899985
f_10      899985
f_11      899986
f_12      899984
f_13      899987
f_14      899986
f_15      899985
f_16      899984
f_17      899986
f_18      899986
f_19           0
f_20           0
f_21           0
f_22           0
f_23           0
f_24           0
f_25           0
f_26           0
f_27      158646
f_28           0
f_29      899998
f_30      899997
target    899998
dtype: int64

📝 Notes:

* f_27 has text with many Dups
* No nan Values. 

* f_29 and f_30 look like classes.

* f_29 [0 1]
* f_30 [0 1 2]

* f_27
text split or other features to extract.

In [8]:
# Function to Clean up the data
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

#reload data

train_data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')

test_data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')



def clean(data):
    
    data = data.drop(['id'], axis = 1)
    
    
    # Split f_27 out by each of the 10 letters. 
    
    
    #This can be done better: 
    data['a1'] = data['f_27'].astype(str).str[0]
    data['a2'] = data['f_27'].astype(str).str[1]
    data['a3'] = data['f_27'].astype(str).str[2]
    data['a4'] = data['f_27'].astype(str).str[3]
    data['a5'] = data['f_27'].astype(str).str[4]
    data['a6'] = data['f_27'].astype(str).str[5]
    data['a7'] = data['f_27'].astype(str).str[6]
    data['a8'] = data['f_27'].astype(str).str[7]
    data['a9'] = data['f_27'].astype(str).str[8]
    data['a10'] = data['f_27'].astype(str).str[9]
 

    #map LabelEncoder for the new f_27 
    
    cols_to_map = ['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10']
    for col in cols_to_map:
        data[col] = le.fit_transform(data[col])

        
    data = data.drop(['f_27'], axis = 1) # features broken out above. No longer needed. 

 
    
     
    
    
        
        
    return data
    
    
    
        
train_data = clean(train_data)
test_data = clean(test_data)

train_data.head()
    
       
y = train_data["target"]
X = train_data.drop("target", axis=1)

    
    
    
    
# X_train.head()
train_data.head()

,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,...,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10
0,-1.373246,0.238887,-0.243376,0.567405,-0.647715,0.839326,0.113133,1,5,1,...,0,1,0,1,3,0,3,1,0,1
1,1.697021,-1.710322,-2.230332,-0.545661,1.113173,-1.552175,0.447825,1,3,4,...,0,2,0,2,2,0,3,2,4,1
2,1.681726,0.616746,-1.027689,0.810492,-0.609086,0.113965,-0.708660,1,0,2,...,0,0,0,4,0,1,2,10,0,3
3,-0.118172,-0.587835,-0.804638,2.086822,0.371005,-0.128831,-0.282575,3,2,1,...,1,3,1,1,0,0,2,1,2,1
4,1.148481,-0.176567,-0.664871,-1.101343,0.467875,0.500117,0.407515,3,3,0,...,1,3,1,2,1,1,2,7,5,4


In [9]:
#train test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)




In [10]:
EPOCHS = 5000

In [11]:
# model = lgb.LGBMClassifier(n_jobs = -1, n_estimators = EPOCHS)
# model.fit(X_train,y_train, eval_set=[(X_test,y_test)], callbacks = [lgb.early_stopping(30)],eval_metric="auc")




In [12]:
# val_preds = model.predict_proba(X_test)
# y_preds = model.predict_proba(X_train)

# print("Intrinsic AUC:", roc_auc_score(y_train, y_preds[:,1]))
# print("Validation AUC:", roc_auc_score(y_test, val_preds[:, 1] ))


In [17]:
##LGBM Binary



lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {
        'task': 'prediction',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'binary_logloss',
        'learning_rate': 0.05,#0.1
        'max_depth': -1,#-1
        'num_leaves': 35,#31
        'min_data_in_leaf': 20,#20
        'bagging_fraction': 1,#1.0
        'bagging_freq': 0,#0
        'feature_fraction': 1,#1.0
        'lambda_l1': 0.8,#0
        'lambda_l2': 0.8,#0
        'num_iterations': 20000,#100
        'verbosity': -1
}



model = lgb.train(
    params,
    train_set=lgb_train,
    valid_sets=lgb_eval,
    early_stopping_rounds=100,
    verbose_eval=100
)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.4296
[200]	valid_0's binary_logloss: 0.360349
[300]	valid_0's binary_logloss: 0.328429
[400]	valid_0's binary_logloss: 0.309229
[500]	valid_0's binary_logloss: 0.297384
[600]	valid_0's binary_logloss: 0.288276
[700]	valid_0's binary_logloss: 0.281484
[800]	valid_0's binary_logloss: 0.275886
[900]	valid_0's binary_logloss: 0.270873
[1000]	valid_0's binary_logloss: 0.267219
[1100]	valid_0's binary_logloss: 0.263624
[1200]	valid_0's binary_logloss: 0.260778
[1300]	valid_0's binary_logloss: 0.257864
[1400]	valid_0's binary_logloss: 0.254953
[1500]	valid_0's binary_logloss: 0.252502
[1600]	valid_0's binary_logloss: 0.250291
[1700]	valid_0's binary_logloss: 0.248136
[1800]	valid_0's binary_logloss: 0.246325
[1900]	valid_0's binary_logloss: 0.244284
[2000]	valid_0's binary_logloss: 0.242091
[2100]	valid_0's binary_logloss: 0.240017
[2200]	valid_0's binary_logloss: 0.238185
[2300]	valid_0's binary_l

In [ ]:
# New pred


sub_preds = model.predict(test_data)

sub_preds.size



In [13]:
test_data.head()

,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,...,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10
0,0.442517,0.174380,-0.999816,0.762741,0.186778,-1.074775,0.501888,6,6,0,...,1,0,0,0,1,0,3,11,0,2
1,-0.605598,-0.305715,0.627667,-0.578898,-1.750931,1.355550,-0.190911,1,3,4,...,0,5,0,1,1,0,4,6,2,1
2,0.303990,2.445110,0.246515,0.818248,0.359731,-1.331845,1.358622,3,3,4,...,1,1,0,2,0,1,1,10,4,4
3,0.154053,0.260126,-1.367092,-0.093175,-1.111034,-0.948481,1.119220,0,0,4,...,0,4,1,4,0,0,2,16,2,2
4,-1.651904,-0.424266,-0.667356,-0.322124,-0.089462,0.181705,1.784983,2,2,2,...,0,4,1,1,1,1,3,0,1,5


In [14]:
#Submission 

# sub_preds = model.predict_proba(test_data)[:, 1]

# sub_preds.size

NameError: name 'model' is not defined

In [ ]:
#check size of df must match

test_ids.size

In [ ]:
#m aking the submissions data frame

df = pd.DataFrame({'id':test_ids.values,
                 "target":sub_preds})

df.head()

In [ ]:
df.to_csv("MayTabPlaySub4.csv", index = False)